In [ ]:
import cv2
import numpy as np
import tkinter as tk
from tkinter import filedialog, messagebox

# ====== Các hàm xử lý ảnh ======
def image_inverse(img):
    return 255 - img

def gamma_correction(img):
    gamma = np.random.uniform(0.5, 2.0)
    look_up = np.array([((i / 255.0) ** gamma) * 255 for i in range(256)]).astype("uint8")
    return cv2.LUT(img, look_up)

def log_transformation(img):
    c = np.random.uniform(1.0, 5.0)
    img_float = np.float32(img) + 1
    log_img = c * np.log(img_float)
    return np.uint8(np.clip(log_img, 0, 255))

def histogram_equalization(img):
    if len(img.shape) == 2:
        return cv2.equalizeHist(img)
    else:
        ycrcb = cv2.cvtColor(img, cv2.COLOR_BGR2YCrCb)
        ycrcb[:, :, 0] = cv2.equalizeHist(ycrcb[:, :, 0])
        return cv2.cvtColor(ycrcb, cv2.COLOR_YCrCb2BGR)

def contrast_stretching(img):
    min_val = np.random.randint(0, 128)
    max_val = np.random.randint(128, 256)
    stretched = (img - min_val) * (255 / (max_val - min_val))
    return np.uint8(np.clip(stretched, 0, 255))

def adaptive_hist_equalization(img):
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    if len(img.shape) == 2:
        return clahe.apply(img)
    else:
        lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
        lab[:, :, 0] = clahe.apply(lab[:, :, 0])
        return cv2.cvtColor(lab, cv2.COLOR_LAB2BGR)

# ====== Hàm chính xử lý và hiển thị ======
def process_images(operation):
    image_paths = ['image1.jpg', 'image2.jpg', 'image3.jpg']
    operation_map = {
        "Inverse": image_inverse,
        "Gamma Correction": gamma_correction,
        "Log Transformation": log_transformation,
        "Histogram Equalization": histogram_equalization,
        "Contrast Stretching": contrast_stretching,
        "Adaptive Histogram Equalization": adaptive_hist_equalization
    }
    for idx, path in enumerate(image_paths):
        img = cv2.imread(path)
        if img is None:
            messagebox.showerror("Lỗi", f"Không thể đọc ảnh {path}")
            continue
        result = operation_map[operation](img)
        cv2.imshow(f"{operation} - Image {idx+1}", result)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# ====== Tạo GUI ======
root = tk.Tk()
root.title("Chọn phương pháp biến đổi ảnh")

options = [
    "Inverse",
    "Gamma Correction",
    "Log Transformation",
    "Histogram Equalization",
    "Contrast Stretching",
    "Adaptive Histogram Equalization"
]

for op in options:
    btn = tk.Button(root, text=op, width=40, command=lambda o=op: process_images(o))
    btn.pack(pady=5)

root.mainloop()
